In [13]:
import os
import pandas as pd

import datasets
from datasets import load_dataset, Dataset, DatasetDict

import torch, numpy
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

os.environ['TRANSFORMERS_OFFLINE'] = '1'
data_path = './data/LCSTS_csv'
source_model = "./hub/mt5-small-finetuned-on-mT5-lcsts"

In [17]:
train_df = pd.read_csv(os.path.join(data_path, 'train.csv'))
eval_df = pd.read_csv(os.path.join(data_path, 'train.csv'))
test_df = pd.read_csv(os.path.join(data_path, 'train.csv'))
# df.columns = ['target', 'text']
# df.to_csv(os.path.join(data_path,'train.csv'))

dataset = Dataset.from_pandas(train_df)


In [23]:
dataset = dataset.remove_columns('Unnamed: 0')

In [24]:
dataset[0]

{'target': '深圳机场9死24伤续：司机全责赔偿或超千万',
 'text': '一辆小轿车，一名女司机，竟造成9死24伤。日前，深圳市交警局对事故进行通报：从目前证据看，事故系司机超速行驶且操作不当导致。目前24名伤员已有6名治愈出院，其余正接受治疗，预计事故赔偿费或超一千万元。'}

In [16]:
dataset = load_dataset("csv", data_files={'train': '{}/train_.csv'.format(data_path), 'validation': '{}/eval.csv'.format(data_path), 'test': '{}/test.csv'.format(data_path)})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset

In [6]:
df.to_csv(os.path.join(data_path,'train.csv'))

In [3]:
df = pd.read_csv(os.path.join(data_path, 't','train.csv'))
df.columns

Index(['Unnamed: 0', 'target', 'text'], dtype='object')

In [6]:
df['target'][0]

'深圳机场9死24伤续：司机全责赔偿或超千万'

In [15]:
# tokenizer = AutoTokenizer.from_pretrained("./hub/mt5-small-finetuned-on-mT5-lcsts", cache_dir = './hub/mt5-small-finetuned-on-mT5-lcsts')
# model = AutoModelForSeq2SeqLM.from_pretrained("./hub/mt5-small-finetuned-on-mT5-lcsts", cache_dir = '.hub/mt5-small-finetuned-on-mT5-lcsts')

tokenizer = AutoTokenizer.from_pretrained("./hub/mt5-small-finetuned-on-lcsts")
model = AutoModelForSeq2SeqLM.from_pretrained("./hub/mt5-small-finetuned-on-lcsts")
# text = '接诊医生为患者测量空腹血糖，血糖条无法自动吸血，导致无法测量，更换新的血糖条后，顺利为患者测量了空腹血糖。'
# label = '医生为患者测量空腹血糖'

In [16]:
max_input_length = 512
max_target_length = 30

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["text"],
        max_length=max_input_length,
        truncation=True,
        padding=True,
    )
    labels = tokenizer(
        examples["target"], 
        max_length=max_target_length, 
        truncation=True,
        padding=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
from datasets import DatasetDict

dataset = DatasetDict.from_csv({'train': '{}/eval.csv'.format(data_path), 'validation': '{}/test.csv'.format(data_path)})
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# 删除原本的列名
tokenized_datasets = tokenized_datasets.remove_columns(dataset["train"].column_names)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Map:   0%|          | 0/10666 [00:00<?, ? examples/s]

from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 8
# Show the training loss with every epoch
logging_steps = len(dataset['train']) // batch_size
model_name = 'mt5'

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-lcsts-cn",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=True,
)

from rouge_chinese import Rouge
import jieba # you can use any other word cutting library

hypothesis = "###刚刚发声，A股这种情况十分罕见！大聪明逆市抄底330亿，一篇研报引爆全球，市场逻辑生变？"
hypothesis = ' '.join(jieba.cut(hypothesis)) 

reference = "刚刚过去的这个月，美股总市值暴跌了将近6万亿美元（折合人民币超过40万亿），这背后的原因可能不仅仅是加息这么简单。最近瑞士信贷知名分析师Zoltan Polzsar撰写了一篇极其重要的文章，详细分析了现有世界秩序的崩坏本质以及美国和西方将要采取的应对策略。在该文中，Zoltan Polzsar直指美国通胀的本质和其长期性。同期，A股市场亦出现了大幅杀跌的情况。"
reference = ' '.join(jieba.cut(reference))

rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)
print(scores)

In [ ]:
import numpy as np
from nltk import sent_tokenize
from rouge_chinese import Rouge

def rouge_score(hypothesis, reference):
    rouge = Rouge()
    return rouge.get_scores(hypothesis, reference)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score(decoded_preds, decoded_labels)
    # result = rouge_score.compute(
    #     predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    # )
    # Extract the median scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

model.train(train_df=train_df, # pandas dataframe with 2 columns: source_text & target_text
            eval_df=eval_df, # pandas dataframe with 2 columns: source_text & target_text
            source_max_token_len = 512, 
            target_max_token_len = 128,
            batch_size = 8,
            max_epochs = 5,
            use_gpu = True,
            outputdir = "outputs",
            early_stopping_patience_epochs = 0,
            precision = 32
            )

In [ ]:
from transformers import Seq2SeqTrainer
os.environ['TRANSFORMERS_OFFLINE'] = '1'

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [19]:
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")

batch_size = 16
train_dataloader = DataLoader(tokenized_datasets["validation"], shuffle=True, collate_fn=data_collator, batch_size=batch_size)
eval_dataloader = DataLoader(tokenized_datasets["validation"], collate_fn=data_collator, batch_size=batch_size)

In [20]:
len(train_dataloader)

69

In [21]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [22]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [30]:
from transformers import get_scheduler

num_train_epochs = 2
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps,)

In [24]:
import nltk

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # ROUGE expects a newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]
    
    return preds[0], labels[0]

In [33]:
import numpy as np
from nltk import sent_tokenize
from rouge_chinese import Rouge
import jieba

class RougeScoreChinese:
    def __init__(self) -> None:
        self.hyp = []
        self.ref = []

        self.rouge = Rouge()

    def add_batch(self, predictions, references):
        predictions = ' '.join(jieba.cut(predictions)) 
        references = ' '.join(jieba.cut(references)) 
        self.hyp.append(predictions)
        self.ref.append(references)

    def compute(self):
        return self.rouge.get_scores(self.hyp, self.ref, avg=True)
    
rouge_score = RougeScoreChinese()

In [ ]:

# Extract the median ROUGE scores
result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
result = {k: round(v, 4) for k, v in result.items()}

In [32]:
import jieba

hypothesis = "刚刚发声，A股这种情况十分罕见！大聪明逆市抄底330亿，一篇研报引爆全球，市场逻辑生变？"
hypothesis = ' '.join(jieba.cut(hypothesis)) 

reference = "刚刚过去的这个月，美股总市值暴跌了将近6万亿美元（折合人民币超过40万亿），这背后的原因可能不仅仅是加息这么简单。最近瑞士信贷知名分析师Zoltan Polzsar撰写了一篇极其重要的文章，详细分析了现有世界秩序的崩坏本质以及美国和西方将要采取的应对策略。在该文中，Zoltan Polzsar直指美国通胀的本质和其长期性。同期，A股市场亦出现了大幅杀跌的情况。"
reference = ' '.join(jieba.cut(reference))

predictions='可穿戴产品的设计原则'
references='可穿戴技术十大设计原则'
# predictions='韩媒:国际油价暴跌让中国笑逐颜开' 
# references='韩媒:油价每下跌10%中国GDP增长0.15%'
# predictions='国务院:禁吸烟广告广告最高罚3万' 
# references='我国拟全面禁止烟草广告影视剧播吸烟最高罚3万'
# predictions='债市打黑风暴:债市打黑背后的“债市打黑'
# references='发改委企业债链条打黑:上百人名单逐个排查'
# predictions='华润万家Ole超市“高端黑”:光鲜背后的乱象横'
# references='曝超市“后厨”:过期食品换包装烂水果切拼盘'
# predictions='韩军军:军方的“最牛”'
# references='朝鲜战争爆发前朝韩军力:朝鲜重火力具有压倒性优势'

rouge = Rouge()
# hyps, refs = [], []
# for _ in range(3):
#     hyps.append(hypothesis)
#     refs.append(reference)
predictions = ' '.join(jieba.cut(hypothesis)) 
references = ' '.join(jieba.cut(reference))
scores = rouge.get_scores(predictions, references)

scores

[{'rouge-1': {'r': 0.08571428571428572,
   'p': 0.2608695652173913,
   'f': 0.1290322543415425},
  'rouge-2': {'r': 0.011235955056179775,
   'p': 0.041666666666666664,
   'f': 0.017699111698645787},
  'rouge-l': {'r': 0.06593406593406594, 'p': 0.24, 'f': 0.10344827248067788}}]

In [13]:
# result = {key: value['p'].mid.fmeasure * 100 for key, value in scores[0].items()}
scores = dict()
for name in ['rouge-1', 'rouge-2', 'rouge-l']:
    # print(1)
    # print(scores[0][name].items())
    result = {k: round(v, 4) for k, v in scores[0][name].items()}
    scores[name] = result
scores

{'rouge-1': {'r': 0.0857, 'p': 0.2609, 'f': 0.129},
 'rouge-2': {'r': 0.0112, 'p': 0.0417, 'f': 0.0177},
 'rouge-l': {'r': 0.0659, 'p': 0.24, 'f': 0.1034}}

In [37]:
from tqdm.auto import tqdm
import torch
import numpy as np

output_dir = "results-mt5-finetuned-squad-accelerate"
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_new_tokens=1000,
            )

            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
            )
            labels = batch["labels"]

            # If we did not pad to max length, we need to pad the labels too
            labels = accelerator.pad_across_processes(
                batch["labels"], dim=1, pad_index=tokenizer.pad_token_id
            )

            generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
            labels = accelerator.gather(labels).cpu().numpy()

            # Replace -100 in the labels as we can't decode them
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens[0]
            decoded_preds = tokenizer.batch_decode(
                generated_tokens, skip_special_tokens=True
            )
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            decoded_preds, decoded_labels = postprocess_text(
                decoded_preds, decoded_labels
            )
            # print('predictions={}, \nreferences={}'.format(decoded_preds, decoded_labels))
            rouge_score.add_batch(predictions=decoded_preds, references=decoded_labels)

    # Compute metrics
    result = rouge_score.compute()
    # print(result)
    # Extract the median ROUGE scores
    # result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    # result = {k: round(v, 4) for k, v in result.items()}

    scores = dict()
    for name in result.keys():
        res = {k: round(v, 4) for k, v in result[name].items()}
        scores[name] = res

    print(f"Epoch {epoch}:", scores)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        # repo.push_to_hub(
        #     commit_message=f"Training in progress epoch {epoch}", blocking=False
        # )

  0%|          | 0/138 [00:00<?, ?it/s]

Epoch 0: {'rouge-1': {'r': 0.2505, 'p': 0.3226, 'f': 0.2739}, 'rouge-2': {'r': 0.1009, 'p': 0.1274, 'f': 0.108}, 'rouge-l': {'r': 0.2322, 'p': 0.2857, 'f': 0.2476}}
Epoch 1: {'rouge-1': {'r': 0.2506, 'p': 0.3225, 'f': 0.274}, 'rouge-2': {'r': 0.1009, 'p': 0.1273, 'f': 0.108}, 'rouge-l': {'r': 0.2322, 'p': 0.2855, 'f': 0.2475}}
